# Laboratorio 1
Edwin Ortega - 22305
Esteban Zambrano - 22119
Juan Pablo Solis - 22102

In [2]:
using JuMP
using HiGHS
using Ipopt
using Optimization

### Ejercicio 1

In [2]:
# Crear el modelo con el solver HiGHS
model = Model(HiGHS.Optimizer)

# Definir variables
@variable(model, x1 >= 0)
@variable(model, x2 >= 0)
@variable(model, x3 >= 0)
@variable(model, x4 >= 0)

# Función objetivo: maximizar z
@objective(model, Max, x1 + x2 + 3x3 + 2x4)

# Restricciones
@constraint(model, x1 + 2x2 - 3x3 + 5x4 <= 4)
@constraint(model, 5x1 - 2x2 + 6x4 <= 8)
@constraint(model, 2x1 + 3x2 - 2x3 + 3x4 <= 3)
@constraint(model, -x1 + x3 + 2x4 <= 0)

# Ejecutar el optimizador
optimize!(model)

# Mostrar resultados
println("Estado de solución: ", termination_status(model))
println("Valor óptimo de z: ", objective_value(model))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))
println("x4 = ", value(x4))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 4 cols; 14 nonzeros
Coefficient ranges:
  Matrix [1e+00, 6e+00]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [3e+00, 8e+00]
Presolving model
4 rows, 3 cols, 10 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-4); elements 0(-14) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  9.0000000000e+00
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Estado de solución: OPTIMAL
Valor óptimo de z: 9.0
x1 = 2.0
x2 = 1.0
x3 = 2.0
x4 = 0.0


### Ejercicio 2 (Modelo de Producción, Períodos múltiples)

#### a) Formular el problema de programamción lineal.

Fue realizado en el documento.


#### b) Resolver el problema

In [8]:
using Printf

# Datos del problema
meses = 1:6
demanda = [180, 250, 190, 140, 220, 250]
costo_produccion = [50, 45, 55, 52, 48, 50]
costo_inventario = [8, 10, 10, 10, 8, 8]
capacidad = 225

# Crear modelo
model = Model(HiGHS.Optimizer)

# Variables: producción x[t] e inventario I[t] para cada mes
@variable(model, x[meses] >= 0)  # producción mensual
@variable(model, I[meses] >= 0)  # inventario final de cada mes

# Restricciones de balance de flujo (inventario)
@constraint(model, x[1] - demanda[1] == I[1])  # I0 = 0
for t in 2:6
    @constraint(model, x[t] + I[t-1] - demanda[t] == I[t])
end

# Restricciones de capacidad de producción
for t in meses
    @constraint(model, x[t] <= capacidad)
end

# Función objetivo: minimizar costo total (producción + inventario)
@objective(model, Min, sum(costo_produccion[t]*x[t] + costo_inventario[t]*I[t] for t in meses))

# Resolver modelo
optimize!(model)

# Mostrar resultados
println("Estado: ", termination_status(model))
println("\nResultado óptimo:")

for t in meses
    @printf("Mes %d:  producir = %.2f,  inventario = %.2f\n", t, value(x[t]), value(I[t]))
end

@printf("\nCosto total mínimo: \$%.2f\n", objective_value(model))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 12 rows; 12 cols; 23 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 8 cols, 10 nonzeros  0s
2 rows, 7 cols, 8 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
2 rows, 7 cols, 8 nonzeros  0s
Presolve : Reductions: rows 2(-10); columns 7(-5); elements 8(-15)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.6295000000e+04 Pr: 2(800) 0s
          3     6.1795000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  6.1795000000e+04
P-D objecti

#### c) Solución con variables enteras

In [ ]:
using Printf

# Datos del problema
meses = 1:6
demanda = [180, 250, 190, 140, 220, 250]
costo_produccion = [50, 45, 55, 52, 48, 50]
costo_inventario = [8, 10, 10, 10, 8, 8]
capacidad = 225

# Crear modelo
model = Model(HiGHS.Optimizer)

# Variables: producción x[t] e inventario I[t] para cada mes
@variable(model, x[meses] >= 0, Int)  # producción mensual
@variable(model, I[meses] >= 0, Int)  # inventario final de cada mes

# Restricciones de balance de flujo (inventario)
@constraint(model, x[1] - demanda[1] == I[1])  # I0 = 0
for t in 2:6
    @constraint(model, x[t] + I[t-1] - demanda[t] == I[t])
end

# Restricciones de capacidad de producción
for t in meses
    @constraint(model, x[t] <= capacidad)
end

# Función objetivo: minimizar costo total (producción + inventario)
@objective(model, Min, sum(costo_produccion[t]*x[t] + costo_inventario[t]*I[t] for t in meses))

# Resolver modelo
optimize!(model)

# Mostrar resultados
println("Estado: ", termination_status(model))
println("\nResultado óptimo:")

for t in meses
    @printf("Mes %d:  producir = %.2f,  inventario = %.2f\n", t, value(x[t]), value(I[t]))
end

@printf("\nCosto total mínimo: \$%.2f\n", objective_value(model))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 12 rows; 12 cols; 23 nonzeros; 12 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 9 cols, 11 nonzeros  0s
2 rows, 6 cols, 7 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQ

### Ejercicio 3 Modelo de asignación de horarios

#### a)

In [3]:
# -----------------------------------------------------
# a) Formulación del PL:
# Variables:
#   x_i >= 0  → autobuses que inician en el turno i
#
# Parámetros:
#   demanda = [4,8,10,7,12,4]
#   cobertura = Dict(
#     1=>[1,2], 2=>[2,3], 3=>[3,4],
#     4=>[4,5], 5=>[5,6], 6=>[6,1]
#   )
#
# Objetivo:
#   min Σ x_i
#
# Restricciones:
#   x1 + x6 >= 4   # intervalo 1
#   x1 + x2 >= 8   # intervalo 2
#   x2 + x3 >=10   # intervalo 3
#   x3 + x4 >= 7   # intervalo 4
#   x4 + x5 >=12   # intervalo 5
#   x5 + x6 >= 4   # intervalo 6
# -----------------------------------------------------

#### b)

In [4]:
# — Datos —
turnos     = 1:6
intervalos = 1:6

cobertura = Dict(
    1 => [1, 2],  # 00:00–07:59
    2 => [2, 3],  # 04:00–11:59
    3 => [3, 4],  # 08:00–15:59
    4 => [4, 5],  # 12:00–19:59
    5 => [5, 6],  # 16:00–23:59
    6 => [6, 1]   # 20:00–03:59
)

demanda = [4, 8, 10, 7, 12, 4]  # según Figura 2

# — Crear modelo con HI‑GHS como solver —
model = Model(HiGHS.Optimizer)

# Variables: x[i] = autobuses que inician en turno i
@variable(model, x[turnos] >= 0)

# Restricciones: cubrir la demanda en cada intervalo
@constraint(model, [j in intervalos],
    sum(x[i] for i in turnos if j in cobertura[i]) >= demanda[j]
)

# Objetivo: minimizar total de autobuses
@objective(model, Min, sum(x[i] for i in turnos))

# Resolver
optimize!(model)

# Resultados
println("Estado de solución: ", termination_status(model))
println("Total de autobuses necesarios: ", objective_value(model))
for i in turnos
    println("  Turno $i → ", value(x[i]) |> round, " autobuses")
end

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 6 cols; 12 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 1e+01]
Presolving model
6 rows, 6 cols, 12 nonzeros  0s
6 rows, 6 cols, 12 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 6(-0); elements 12(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(45) 0s
          5     2.6000000000e+01 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 5
Objective value     :  2.6000000000e+01
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Estado de solución: OPTIMAL
Total de autobuses necesarios: 26.0
  Turno 1 → 4.0 autobuses
  Turno 2 → 4.0 autobuses
  Turno 3 → 6.0 autobuses
  Turno 4 → 1.0 autobuses
  Turno 5 → 11.0 a

#### **Ejercicio 4**

#### **Inciso A: Programacion Lineal**

In [ ]:
# -----------------------------------------------------
# a) Formulación del PL:
# Variables:
#   x1 >= 0 → unidades sencillas
#   x2 >= 0 → unidades dobles
#   x3 >= 0 → unidades triples
#   x4 >= 0 → unidades cuádruples
#
# Parámetros:
#   Área total disponible = 300 * 0.25 = 75 acres
#   Área útil = 75 * 0.85 = 63.75 acres
#
#   Tamaños de lote (acres): 
#     sencilla = 0.18, doble = 0.28, triple = 0.4, cuádruple = 0.5
#
#   Costo construcción por unidad:
#     sencilla = $50,000, doble = $70,000, triple = $130,000, cuádruple = $160,000
#   Presupuesto disponible: $15,000,000
#
#   Impuesto recaudado por unidad:
#     sencilla = $1,000, doble = $1,900, triple = $2,700, cuádruple = $3,400
#
# Objetivo:
#   max Z = 1000·x1 + 1900·x2 + 2700·x3 + 3400·x4
#
# Restricciones:
#   0.18·x1 + 0.28·x2 + 0.4·x3 + 0.5·x4 ≤ 63.75          # límite de área
#   50000·x1 + 70000·x2 + 130000·x3 + 160000·x4 ≤ 15e6   # presupuesto máximo
#   x3 + x4 ≥ 0.25·(x1 + x2 + x3 + x4)                   # mín. 25% triples+cuádruples
#   x1 ≥ 0.20·(x1 + x2 + x3 + x4)                        # mín. 20% sencillas
#   x2 ≥ 0.10·(x1 + x2 + x3 + x4)                        # mín. 10% dobles
# -----------------------------------------------------


#### **Inciso B: Programacion Variables Continuas**

In [ ]:
using JuMP
using HiGHS  
modelo = Model(HiGHS.Optimizer)

# Variables de decisión (continuas)
@variable(modelo, x1 >= 0)  # Unidades sencillas
@variable(modelo, x2 >= 0)  # Unidades dobles
@variable(modelo, x3 >= 0)  # Unidades triples
@variable(modelo, x4 >= 0)  # Unidades cuádruples

# Función objetivo: maximizar recaudación
@objective(modelo, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

# Restricciones
@constraint(modelo, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)  # área disponible
@constraint(modelo, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)  # presupuesto
@constraint(modelo, x3 + x4 >= 0.25 * (x1 + x2 + x3 + x4))  # ≥25% triples/cuádruples
@constraint(modelo, x1 >= 0.20 * (x1 + x2 + x3 + x4))       # ≥20% sencillas
@constraint(modelo, x2 >= 0.10 * (x1 + x2 + x3 + x4))       # ≥10% dobles


optimize!(modelo)

# Mostrar resultados
println("Estado: ", termination_status(modelo))
println("Valor óptimo (recaudación): \$", objective_value(modelo))
println("Unidades:")
println("  Sencillas:   ", value(x1))
println("  Dobles:      ", value(x2))
println("  Triples:     ", value(x3))
println("  Cuádruples:  ", value(x4))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 5 rows; 4 cols; 20 nonzeros
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 2e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Presolve : Reductions: rows 5(-0); columns 4(-0); elements 20(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.2578114611e+02 Ph1: 4(14.302); Du: 4(125.781) 0s
          3     3.5555555556e+05 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  3.5555555556e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Estado: OPTIMAL
Valor óptimo (recaudación): $355555.55555555556
Unidades:
  Sencillas:   37.03703703703704
  Dobles:      101.85185185185185
  Triples:     46.296296

#### **Inciso C: Programacion Enteras**

In [ ]:
using JuMP
using HiGHS

modelo_entero = Model(HiGHS.Optimizer)

# Variables de decisión enteras
@variable(modelo_entero, x1 >= 0, Int)  # Unidades sencillas
@variable(modelo_entero, x2 >= 0, Int)  # Unidades dobles
@variable(modelo_entero, x3 >= 0, Int)  # Unidades triples
@variable(modelo_entero, x4 >= 0, Int)  # Unidades cuádruples

# Función objetivo: maximizar recaudación
@objective(modelo_entero, Max, 1000x1 + 1900x2 + 2700x3 + 3400x4)

# Restricciones
@constraint(modelo_entero, 0.18x1 + 0.28x2 + 0.4x3 + 0.5x4 <= 63.75)  # área
@constraint(modelo_entero, 50000x1 + 70000x2 + 130000x3 + 160000x4 <= 15_000_000)  # presupuesto
@constraint(modelo_entero, x3 + x4 >= 0.25 * (x1 + x2 + x3 + x4))  # ≥25% triples/cuádruples
@constraint(modelo_entero, x1 >= 0.20 * (x1 + x2 + x3 + x4))       # ≥20% sencillas
@constraint(modelo_entero, x2 >= 0.10 * (x1 + x2 + x3 + x4))       # ≥10% dobles

# Resolver
optimize!(modelo_entero)

# Resultados
println("Estado: ", termination_status(modelo_entero))
println("Valor óptimo (recaudación): \$", objective_value(modelo_entero))
println("Unidades:")
println("  Sencillas:   ", value(x1))
println("  Dobles:      ", value(x2))
println("  Triples:     ", value(x3))
println("  Cuádruples:  ", value(x4))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 5 rows; 4 cols; 20 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e-01, 2e+05]
  Cost   [1e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 2e+07]
Presolving model
5 rows, 4 cols, 20 nonzeros  0s
5 rows, 4 cols, 20 nonzeros  0s
Objective function is integral with scale 0.01

Solving MIP model with:
   5 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous, 0 domain fixed)
   20 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |   

#### **Interpretacion**

Al resolver el modelo de programación lineal con variables continuas se obtuvo una recaudación óptima de $355,556, construyendo aproximadamente 37 unidades sencillas, 102 dobles, 46 triples y ninguna cuádruple. Sin embargo, al aplicar el modelo con variables enteras, la recaudación óptima fue ligeramente menor, alcanzando $355,500, con una solución factible de 36 unidades sencillas, 99 dobles, 31 triples y 14 cuádruples. Aunque la diferencia en ingresos es mínima ($56), la solución entera respeta la naturaleza real del problema, al asignar cantidades enteras de viviendas cumpliendo con las restricciones proporcionales y presupuestarias